In [2]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

# Load and prepare data
df = pd.read_csv('dpt2020.csv', sep=';')
df['annais'] = pd.to_numeric(df['annais'], errors='coerce').astype('Int64')
df = df[(df['annais'] >= 1900) & (df['annais'] <= 2020)]

# Group by year and name and sum the birth counts
grouped_df = df.groupby(['annais', 'preusuel'])['nombre'].sum().reset_index()

# Function to get the top 15 names per year
def get_top_15(df):
    top_15 = df.sort_values(by='nombre', ascending=False).groupby('annais').head(15)
    return top_15

# Apply function to get the top 15 names per year
top_15_df = get_top_15(grouped_df)

# Pivot the data for the heatmap
heatmap_data = top_15_df.pivot(index='preusuel', columns='annais', values='nombre').fillna(0)

# Initialize Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1('Top 15 Most Popular Names Each Year (1900-2020)', style={'text-align': 'center'}),
    dcc.Graph(id='heatmap',
              figure={
                  'data': [go.Heatmap(
                      z=heatmap_data.values,
                      x=heatmap_data.columns,
                      y=heatmap_data.index,
                      colorscale='Viridis')],
                  'layout': go.Layout(
                      xaxis=dict(title='Year'),
                      yaxis=dict(title='Name'),
                      margin=dict(l=150, r=10, t=40, b=30),
                      height=800
                  )
              })
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
